In [3]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('pokemon.csv')

In [4]:
image_folder = 'images'

def get_image_path(pokemon_id):
    path = os.path.join(image_folder, f'{pokemon_id}.png')
    return path if os.path.exists(path) else None

In [5]:
df['image_path'] = df['Name'].apply(get_image_path)

In [8]:
# Check for missing images
missing = df[df['image_path'].isna()]
if not missing.empty:
    print(f"Warning: {len(missing)} images not found")
    print(missing['Name'].tolist())
    

In [9]:
print(df.head())

         Name  Type1   Type2   Evolution             image_path
0   bulbasaur  Grass  Poison     ivysaur   images\bulbasaur.png
1     ivysaur  Grass  Poison    venusaur     images\ivysaur.png
2    venusaur  Grass  Poison         NaN    images\venusaur.png
3  charmander   Fire     NaN  charmeleon  images\charmander.png
4  charmeleon   Fire     NaN   charizard  images\charmeleon.png


In [ ]:
# Import libraries for image processing
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Check a sample image - original size
sample_path = df['image_path'].iloc[0]
sample_img = Image.open(sample_path)
print(f"Original image mode: {sample_img.mode}")
print(f"Original image size: {sample_img.size}")
print(f"Original shape as array: {np.array(sample_img).shape}")
print(f"\nWill resize to: (224, 224) for EfficientNet-B0")
sample_img

In [ ]:
def load_image_as_rgb(image_path, target_size=(224, 224)):
    """
    Load an image, convert RGBA to RGB, and resize for EfficientNet-B0.
    
    Args:
        image_path: Path to the image file
        target_size: Tuple of (width, height) - default 224x224 for EfficientNet-B0
        
    Returns:
        numpy array of shape (224, 224, 3) with RGB values
    """
    img = Image.open(image_path)
    
    # Convert RGBA to RGB if necessary
    if img.mode == 'RGBA':
        # Create a white background
        rgb_img = Image.new('RGB', img.size, (255, 255, 255))
        # Paste the RGBA image on white background using alpha channel as mask
        rgb_img.paste(img, mask=img.split()[3])  # 3 is the alpha channel
        img = rgb_img
    elif img.mode != 'RGB':
        # Convert any other mode to RGB
        img = img.convert('RGB')
    
    # Resize to target size for EfficientNet-B0 (224x224)
    img = img.resize(target_size, Image.Resampling.LANCZOS)
    
    # Convert to numpy array
    return np.array(img)

In [ ]:
# Test the conversion and resizing on sample image
rgb_array = load_image_as_rgb(sample_path)
print(f"Resized RGB array shape: {rgb_array.shape}")  # Should be (224, 224, 3)
print(f"Data type: {rgb_array.dtype}")
print(f"Value range: [{rgb_array.min()}, {rgb_array.max()}]")

# Display the converted and resized image
plt.imshow(rgb_array)
plt.title(f"{df['Name'].iloc[0]} - Type: {df['Type1'].iloc[0]}\n(Resized to 224x224 for EfficientNet-B0)")
plt.axis('off')
plt.show()

In [ ]:
# Load all images and convert to RGB arrays
print("Loading all images...")
image_arrays = []
valid_indices = []

for idx, row in df.iterrows():
    image_path = row['image_path']
    if pd.notna(image_path) and os.path.exists(image_path):
        try:
            rgb_array = load_image_as_rgb(image_path)
            image_arrays.append(rgb_array)
            valid_indices.append(idx)
        except Exception as e:
            print(f"Error loading {row['Name']}: {e}")
    
print(f"\nSuccessfully loaded {len(image_arrays)} images")

# Filter dataframe to only include successfully loaded images
df_valid = df.loc[valid_indices].reset_index(drop=True)

In [ ]:
# Verify all images are now standardized to (224, 224, 3)
shapes = [arr.shape for arr in image_arrays]
unique_shapes = set(shapes)

print(f"Number of unique image shapes: {len(unique_shapes)}")
print(f"All images shape: {shapes[0] if shapes else 'None'}")
print(f"Total images loaded: {len(image_arrays)}")

# Verify they're all (224, 224, 3)
all_correct_shape = all(shape == (224, 224, 3) for shape in shapes)
print(f"\n✅ All images are (224, 224, 3): {all_correct_shape}")

In [ ]:
# Visualize a sample of converted images
fig, axes = plt.subplots(3, 4, figsize=(12, 9))
axes = axes.ravel()

for i in range(12):
    if i < len(image_arrays):
        axes[i].imshow(image_arrays[i])
        axes[i].set_title(f"{df_valid['Name'].iloc[i]}\n{df_valid['Type1'].iloc[i]}", 
                         fontsize=9)
        axes[i].axis('off')

plt.tight_layout()
plt.suptitle('Sample Pokemon Images (RGBA → RGB)', y=1.02, fontsize=14)
plt.show()

## Summary

✅ **Image Conversion & Preprocessing Complete**
- Loaded images from PNG files
- Converted RGBA → RGB (using white background for transparency)
- **Resized all images to 224×224 for EfficientNet-B0**
- Created standardized multidimensional numpy arrays (224 × 224 × 3)
- Verified all images loaded successfully with correct shape

**Why 224×224?**
- EfficientNet-B0 requires 224×224 input
- Used LANCZOS resampling for high-quality resizing
- All images now have consistent dimensions

**Next Steps:**
1. Normalize pixel values (divide by 255 or use ImageNet stats)
2. Create train/val/test splits (70/10/20)
3. Set up data augmentation pipeline
4. Export preprocessed data for training